# Riskfolio-Lib Tutorial: 
<br>__[Financionerioncios](https://financioneroncios.wordpress.com)__
<br>__[Orenji](https://www.orenj-i.net)__
<br>__[Riskfolio-Lib](https://riskfolio-lib.readthedocs.io/en/latest/)__
<br>__[Dany Cajas](https://www.linkedin.com/in/dany-cajas/)__
<a href='https://ko-fi.com/B0B833SXD' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://cdn.ko-fi.com/cdn/kofi1.png?v=2' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a> 

## Tutorial 4: Bond Portfolio Optimization and Immunization

If you want to know more about the mathematics behind this model you can check the following posts:  __[Valorización de Bonos con Python parte II](https://financioneroncios.wordpress.com/2018/05/23/valorizacion-de-bonos-con-python-parte-ii/)__, __[Fixed Income Portfolio Optimization with Python](https://financioneroncios.wordpress.com/2020/01/09/fixed-income-portfolio-optimization-with-python/)__ 

## 1. Uploading the data:

In [1]:
########################################################################
# Uploading Data
########################################################################

import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# Interest Rates Data
kr = pd.read_excel('KeyRates.xlsx', engine='openpyxl', index_col=0, header=0)/100

# Prices  Data
assets = pd.read_excel('Assets.xlsx', engine='openpyxl', index_col=0, header=0)

# Find common dates
a = pd.merge(left=assets, right=kr, how='inner', on='Date')
dates = a.index

# Calculate interest rates returns
kr_returns = kr.loc[dates,:].sort_index().diff().dropna()
kr_returns.sort_index(ascending=False, inplace=True)

# List of instruments
equity = ['APA','CMCSA','CNP','HPQ','PSA','SEE','ZION']
bonds = ['PEP11900D031', 'PEP13000D012', 'PEP13000M088',
         'PEP23900M103','PEP70101M530','PEP70101M571',
         'PEP70310M156']

# Calculate assets returns
assets_returns = assets.loc[dates, equity + bonds]
assets_returns = assets_returns.sort_index().pct_change().dropna()
assets_returns.sort_index(ascending=False, inplace=True)

# Show tables
display(kr_returns.head().style.format("{:.4%}"))
display(assets_returns.head().style.format("{:.4%}"))

,0,90,180,360,720,1800,3600,7200,10800
Date,,,,,,,,,
2017-11-16 00:00:00,0.0000%,0.0059%,0.0108%,0.0178%,0.0246%,0.0213%,0.0075%,-0.0048%,-0.0093%
2017-11-15 00:00:00,0.0180%,0.0247%,0.0303%,0.0391%,0.0495%,0.0558%,0.0512%,0.0450%,0.0417%
2017-11-14 00:00:00,-0.1800%,-0.1710%,-0.1624%,-0.1460%,-0.1167%,-0.0506%,0.0140%,0.0676%,0.0861%
2017-11-13 00:00:00,0.0000%,0.0013%,0.0025%,0.0048%,0.0088%,0.0174%,0.0258%,0.0334%,0.0364%
2017-11-10 00:00:00,0.0000%,0.0026%,0.0043%,0.0054%,0.0017%,-0.0248%,-0.0615%,-0.0936%,-0.1054%


,APA,CMCSA,CNP,HPQ,PSA,SEE,ZION,PEP11900D031,PEP13000D012,PEP13000M088,PEP23900M103,PEP70101M530,PEP70101M571,PEP70310M156
Date,,,,,,,,,,,,,,
2017-11-16 00:00:00,-1.3161%,-0.2958%,-1.0903%,0.9831%,1.7234%,1.4016%,-0.8387%,-0.0411%,-0.0380%,-0.0597%,-0.0737%,-0.0116%,0.0076%,-0.0633%
2017-11-15 00:00:00,-2.0296%,0.8682%,-1.1202%,0.0000%,-1.3479%,-0.3326%,0.0215%,-0.1626%,-0.3076%,-0.3041%,-0.2286%,-0.4459%,-0.4651%,-0.2146%
2017-11-14 00:00:00,-3.7020%,-1.0470%,1.0800%,0.8975%,-0.1548%,0.2668%,2.6950%,0.2320%,0.0236%,0.1040%,0.2373%,-0.2741%,-0.3932%,0.2151%
2017-11-13 00:00:00,-1.4503%,1.0855%,0.7480%,-0.2826%,0.8179%,1.4205%,3.4145%,0.0906%,0.0064%,-0.0767%,0.0354%,-0.0835%,-0.1114%,-0.0081%
2017-11-10 00:00:00,-2.4536%,0.7932%,-1.3418%,-0.5155%,0.0710%,-0.9381%,-0.0910%,0.1194%,0.3792%,0.3047%,0.1355%,0.7118%,0.8207%,-0.0090%


In [2]:
########################################################################
# Uploading Duration and Convexity Matrixes
########################################################################

durations = pd.read_excel('durations.xlsx', index_col=0, header=0)
convexity = pd.read_excel('convexity.xlsx', index_col=0, header=0)

print('Durations Matrix')
display(durations.head().style.format("{:.4f}").background_gradient(cmap='YlGn'))
print('')
print('Convexity Matrix')
display(convexity.head().style.format("{:.4f}").background_gradient(cmap='YlGn'))

Durations Matrix


,R 0,R 90,R 180,R 360,R 720,R 1800,R 3600,R 7200,R 10800
PEP11900D031,0.0012,0.0057,0.0192,0.0730,0.3685,3.0416,0.0030,0.0000,0.0000
PEP13000D012,0.0000,0.0078,0.0142,0.0617,0.3327,1.0902,4.8055,0.2074,0.0000
PEP13000M088,0.0013,0.0004,0.0147,0.0501,0.2770,2.4626,3.0764,0.0000,0.0000
PEP23900M103,0.0000,0.0005,0.0117,0.0405,0.2274,3.9726,0.0381,0.0000,0.0000
PEP70101M530,0.0000,0.0052,0.0101,0.0442,0.2488,0.8826,4.9147,3.5537,0.0000



Convexity Matrix


,R^2 0,R^2 90,R^2 180,R^2 360,R^2 720,R^2 1800,R^2 3600,R^2 7200,R^2 10800
PEP11900D031,0.0004,0.0032,0.0167,0.0928,0.7741,15.5617,0.0000,0.0000,0.0000
PEP13000D012,0.0000,0.0057,0.0070,0.0756,0.7210,4.4984,45.2159,0.1105,0.0000
PEP13000M088,0.0010,0.0001,0.0192,0.0736,0.6161,8.8479,16.2880,0.0000,0.0000
PEP23900M103,0.0000,0.0000,0.0156,0.0644,0.5161,22.1272,0.0022,0.0000,0.0000
PEP70101M530,0.0000,0.0038,0.0052,0.0561,0.5530,3.7373,38.2315,26.1464,0.0000


## 2. Estimating Mean Variance Portfolio

### 2.1 Building the loadings matrix and risk factors returns.

This part shows how to build a personalized loadings matrix that will be used by __Riskfolio-Lib__ to calculate the expected returns and covariance matrix.

In [3]:
########################################################################
# Building The Loadings Matrix
########################################################################

loadings = pd.concat([-1.0 * durations, 0.5 * convexity], axis = 1)

display(loadings.style.format("{:.4f}").background_gradient(cmap='YlGn'))

,R 0,R 90,R 180,R 360,R 720,R 1800,R 3600,R 7200,R 10800,R^2 0,R^2 90,R^2 180,R^2 360,R^2 720,R^2 1800,R^2 3600,R^2 7200,R^2 10800
PEP11900D031,-0.0012,-0.0057,-0.0192,-0.0730,-0.3685,-3.0416,-0.0030,-0.0000,-0.0000,0.0002,0.0016,0.0083,0.0464,0.3871,7.7809,0.0000,0.0000,0.0000
PEP13000D012,-0.0000,-0.0078,-0.0142,-0.0617,-0.3327,-1.0902,-4.8055,-0.2074,-0.0000,0.0000,0.0029,0.0035,0.0378,0.3605,2.2492,22.6080,0.0553,0.0000
PEP13000M088,-0.0013,-0.0004,-0.0147,-0.0501,-0.2770,-2.4626,-3.0764,-0.0000,-0.0000,0.0005,0.0000,0.0096,0.0368,0.3081,4.4240,8.1440,0.0000,0.0000
PEP23900M103,-0.0000,-0.0005,-0.0117,-0.0405,-0.2274,-3.9726,-0.0381,-0.0000,-0.0000,0.0000,0.0000,0.0078,0.0322,0.2581,11.0636,0.0011,0.0000,0.0000
PEP70101M530,-0.0000,-0.0052,-0.0101,-0.0442,-0.2488,-0.8826,-4.9147,-3.5537,-0.0000,0.0000,0.0019,0.0026,0.0280,0.2765,1.8686,19.1157,13.0732,0.0000
PEP70101M571,-0.0015,-0.0039,-0.0126,-0.0501,-0.2829,-1.0108,-2.5878,-6.0312,-0.4501,0.0002,0.0016,0.0064,0.0319,0.3123,2.1336,10.1632,49.9021,0.4523
PEP70310M156,-0.0000,-0.0039,-0.0097,-0.0403,-0.2614,-3.8920,-0.0000,-0.0000,-0.0000,0.0000,0.0010,0.0030,0.0268,0.2508,10.6813,0.0000,0.0000,0.0000


In [4]:
########################################################################
# Building the risk factors returns matrix
########################################################################

kr_returns_2 =  kr_returns ** 2
cols = loadings.columns

X = pd.concat([kr_returns, kr_returns_2], axis=1)
X.columns = cols

display(X.head().style.format("{:.4%}"))

,R 0,R 90,R 180,R 360,R 720,R 1800,R 3600,R 7200,R 10800,R^2 0,R^2 90,R^2 180,R^2 360,R^2 720,R^2 1800,R^2 3600,R^2 7200,R^2 10800
Date,,,,,,,,,,,,,,,,,,
2017-11-16 00:00:00,0.0000%,0.0059%,0.0108%,0.0178%,0.0246%,0.0213%,0.0075%,-0.0048%,-0.0093%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
2017-11-15 00:00:00,0.0180%,0.0247%,0.0303%,0.0391%,0.0495%,0.0558%,0.0512%,0.0450%,0.0417%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
2017-11-14 00:00:00,-0.1800%,-0.1710%,-0.1624%,-0.1460%,-0.1167%,-0.0506%,0.0140%,0.0676%,0.0861%,0.0003%,0.0003%,0.0003%,0.0002%,0.0001%,0.0000%,0.0000%,0.0000%,0.0001%
2017-11-13 00:00:00,0.0000%,0.0013%,0.0025%,0.0048%,0.0088%,0.0174%,0.0258%,0.0334%,0.0364%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
2017-11-10 00:00:00,0.0000%,0.0026%,0.0043%,0.0054%,0.0017%,-0.0248%,-0.0615%,-0.0936%,-0.1054%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0001%,0.0001%


In [5]:
########################################################################
# Building the asset returns matrix
########################################################################

Y = assets_returns[loadings.index]

display(Y.head())

,PEP11900D031,PEP13000D012,PEP13000M088,PEP23900M103,PEP70101M530,PEP70101M571,PEP70310M156
Date,,,,,,,
2017-11-16,-0.000411,-0.000380,-0.000597,-0.000737,-0.000116,0.000076,-0.000633
2017-11-15,-0.001626,-0.003076,-0.003041,-0.002286,-0.004459,-0.004651,-0.002146
2017-11-14,0.002320,0.000236,0.001040,0.002373,-0.002741,-0.003932,0.002151
2017-11-13,0.000906,0.000064,-0.000767,0.000354,-0.000835,-0.001114,-0.000081
2017-11-10,0.001194,0.003792,0.003047,0.001355,0.007118,0.008207,-0.000090


### 2.2 Calculating the portfolio that maximizes Sharpe ratio.

In [6]:
########################################################################
# Calculating optimum portfolio
########################################################################

import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

port.factors = X
port.factors_stats(method_mu=method_mu, method_cov=method_cov, d=0.94, B=loadings)

# Estimate optimal portfolio:

model='FM' # Factor Model
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.style.format("{:.4%}").background_gradient(cmap='YlGn'))

,weights
PEP11900D031,0.0000%
PEP13000D012,6.6695%
PEP13000M088,0.0000%
PEP23900M103,0.0000%
PEP70101M530,32.3828%
PEP70101M571,60.9477%
PEP70310M156,0.0000%


## 3. Optimization with Key Rate Durations Constraints

This part shows how __Riskfolio-Lib__ can be used to build immunized portfolios using __duration matching__ and __convexity matching__, however the example only use duration matching. More information about inmunization theory can be found in this __[link](https://www.investopedia.com/terms/i/immunization.asp)__.

### 3.1 Statistics of Risk Factors

In [7]:
########################################################################
# Displaying factors statistics
########################################################################

table = pd.concat([loadings.min(), loadings.max()], axis=1)
table.columns = ['min', 'max']
display(table.iloc[:9,:].style.format("{:.4f}").background_gradient(cmap='YlGn'))
display(X.iloc[:,:9].corr().style.format("{:.4f}").background_gradient(cmap='YlGn'))

,min,max
R 0,-0.0015,-0.0000
R 90,-0.0078,-0.0004
R 180,-0.0192,-0.0097
R 360,-0.0730,-0.0403
R 720,-0.3685,-0.2274
R 1800,-3.9726,-0.8826
R 3600,-4.9147,-0.0000
R 7200,-6.0312,-0.0000
R 10800,-0.4501,-0.0000


,R 0,R 90,R 180,R 360,R 720,R 1800,R 3600,R 7200,R 10800
R 0,1.0000,0.9233,0.7183,0.2411,-0.0500,0.0302,0.0256,0.0284,0.0489
R 90,0.9233,1.0000,0.9197,0.5137,0.0981,0.0442,0.0441,0.0441,0.0523
R 180,0.7183,0.9197,1.0000,0.7901,0.3509,0.1002,0.0579,0.0700,0.0732
R 360,0.2411,0.5137,0.7901,1.0000,0.7887,0.2784,0.0751,0.1126,0.1252
R 720,-0.0500,0.0981,0.3509,0.7887,1.0000,0.6022,0.1808,0.1729,0.1952
R 1800,0.0302,0.0442,0.1002,0.2784,0.6022,1.0000,0.7349,0.4609,0.3664
R 3600,0.0256,0.0441,0.0579,0.0751,0.1808,0.7349,1.0000,0.8102,0.6100
R 7200,0.0284,0.0441,0.0700,0.1126,0.1729,0.4609,0.8102,1.0000,0.9189
R 10800,0.0489,0.0523,0.0732,0.1252,0.1952,0.3664,0.6100,0.9189,1.0000


### 3.2 Creating Constraints on Key Rate Durations

In this example we are going to put a limit on the maximum duration that the portfolio can reach. The key rate durations of portfolio for 1800, 3600 and 7200 days will be lower than -2, -2 and -3.

In [8]:
########################################################################
# Creating durations constraints
########################################################################

constraints = {'Disabled': [False, False, False],
               'Factor': ['R 1800', 'R 3600', 'R 7200'],
               'Sign': ['<=', '<=', '<='],
               'Value': [-2, -2, -3],
               'Relative Factor': ['', '', '']}

constraints = pd.DataFrame(constraints)

display(constraints)

,Disabled,Factor,Sign,Value,Relative Factor
0,False,R 1800,<=,-2,
1,False,R 3600,<=,-2,
2,False,R 7200,<=,-3,


### 3.3 Estimating Optimum Portfolio with Key Rate Durations Constraints

In [9]:
########################################################################
# Estimating optimum portfolio with key rate duration constraints
########################################################################

C, D = rp.factors_constraints(constraints, loadings)

port.ainequality = C
port.binequality = D

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.style.format("{:.4%}").background_gradient(cmap='YlGn'))

,weights
PEP11900D031,3.4877%
PEP13000D012,0.0000%
PEP13000M088,0.8050%
PEP23900M103,16.4913%
PEP70101M530,18.7684%
PEP70101M571,45.0956%
PEP70310M156,15.3520%


We can see that with this constraints the weights of the portfolio are more spread along all assets. To show that the portfolio full fill all constraints we will calculate the sensitivities of the portfolio.

In [10]:
########################################################################
# Calculating portfolio sensitivities for each risk factor
########################################################################

d_ = np.matrix(loadings).T * np.matrix(w)
d_ = pd.DataFrame(d_, index=loadings.columns, columns=['Values'])

display(d_.style.format("{:.4f}").background_gradient(cmap='YlGn'))

,Values
R 0,-0.0007
R 90,-0.0036
R 180,-0.0118
R 360,-0.0467
R 720,-0.2670
R 1800,-2.0000
R 3600,-2.1206
R 7200,-3.3868
R 10800,-0.2030
R^2 0,0.0001


## 4. Estimating Mean Variance Portfolio

### 4.1 Building the loadings matrix and risk factors returns.

This part shows how to build a personalized loadings matrix that will be used by __Riskfolio-Lib__ to calculate the expected returns and covariance matrix.

In [11]:
########################################################################
# Building the risk factors returns matrix
########################################################################

# Removing bond returns from factors matrix
cols = assets_returns.columns
cols = ~cols.isin(loadings.index)
cols = assets_returns.columns[cols]

# Other approach for removing bond returns from factors matrix
cols = [col for col in assets_returns.columns if col not in loadings.index]

X = pd.concat([assets_returns[cols], X], axis=1)

display(X.head())

,APA,CMCSA,CNP,HPQ,PSA,SEE,ZION,R 0,R 90,R 180,...,R 10800,R^2 0,R^2 90,R^2 180,R^2 360,R^2 720,R^2 1800,R^2 3600,R^2 7200,R^2 10800
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-16,-0.013161,-0.002958,-0.010903,0.009831,0.017234,0.014016,-0.008387,0.00000,0.000059,0.000108,...,-0.000093,0.000000e+00,3.527647e-09,1.163831e-08,3.181621e-08,6.048648e-08,4.552420e-08,5.573518e-09,2.314283e-09,8.695936e-09
2017-11-15,-0.020296,0.008682,-0.011202,0.000000,-0.013479,-0.003326,0.000215,0.00018,0.000247,0.000303,...,0.000417,3.225005e-08,6.082093e-08,9.197391e-08,1.529780e-07,2.448864e-07,3.108575e-07,2.620815e-07,2.025117e-07,1.736147e-07
2017-11-14,-0.037020,-0.010470,0.010800,0.008975,-0.001548,0.002668,0.026950,-0.00180,-0.001710,-0.001624,...,0.000861,3.241235e-06,2.925701e-06,2.636999e-06,2.131988e-06,1.362113e-06,2.555747e-07,1.966922e-08,4.572289e-07,7.417774e-07
2017-11-13,-0.014503,0.010855,0.007480,-0.002826,0.008179,0.014205,0.034145,0.00000,0.000013,0.000025,...,0.000364,0.000000e+00,1.681950e-10,6.400900e-10,2.324301e-09,7.748577e-09,3.033345e-08,6.657174e-08,1.116368e-07,1.328391e-07
2017-11-10,-0.024536,0.007932,-0.013418,-0.005155,0.000710,-0.009381,-0.000910,0.00000,0.000026,0.000043,...,-0.001054,0.000000e+00,6.770924e-10,1.820644e-09,2.881542e-09,2.791573e-10,6.131368e-08,3.779975e-07,8.759856e-07,1.110697e-06


In [12]:
########################################################################
# Building the asset returns matrix
########################################################################

Y = pd.concat([assets_returns[cols], Y], axis=1)

display(Y.head())

,APA,CMCSA,CNP,HPQ,PSA,SEE,ZION,PEP11900D031,PEP13000D012,PEP13000M088,PEP23900M103,PEP70101M530,PEP70101M571,PEP70310M156
Date,,,,,,,,,,,,,,
2017-11-16,-0.013161,-0.002958,-0.010903,0.009831,0.017234,0.014016,-0.008387,-0.000411,-0.000380,-0.000597,-0.000737,-0.000116,0.000076,-0.000633
2017-11-15,-0.020296,0.008682,-0.011202,0.000000,-0.013479,-0.003326,0.000215,-0.001626,-0.003076,-0.003041,-0.002286,-0.004459,-0.004651,-0.002146
2017-11-14,-0.037020,-0.010470,0.010800,0.008975,-0.001548,0.002668,0.026950,0.002320,0.000236,0.001040,0.002373,-0.002741,-0.003932,0.002151
2017-11-13,-0.014503,0.010855,0.007480,-0.002826,0.008179,0.014205,0.034145,0.000906,0.000064,-0.000767,0.000354,-0.000835,-0.001114,-0.000081
2017-11-10,-0.024536,0.007932,-0.013418,-0.005155,0.000710,-0.009381,-0.000910,0.001194,0.003792,0.003047,0.001355,0.007118,0.008207,-0.000090


In [13]:
########################################################################
# Building The Loadings Matrix
########################################################################

a = np.identity(len(cols))
a = pd.DataFrame(a, index=cols, columns=cols)
loadings = pd.concat([a, loadings], axis = 1)
loadings.fillna(0, inplace=True)

display(loadings.style.format("{:.4f}").background_gradient(cmap='YlGn'))

,APA,CMCSA,CNP,HPQ,PSA,SEE,ZION,R 0,R 90,R 180,R 360,R 720,R 1800,R 3600,R 7200,R 10800,R^2 0,R^2 90,R^2 180,R^2 360,R^2 720,R^2 1800,R^2 3600,R^2 7200,R^2 10800
APA,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
CMCSA,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
CNP,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
HPQ,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
PSA,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
SEE,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
ZION,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
PEP11900D031,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0012,-0.0057,-0.0192,-0.0730,-0.3685,-3.0416,-0.0030,-0.0000,-0.0000,0.0002,0.0016,0.0083,0.0464,0.3871,7.7809,0.0000,0.0000,0.0000
PEP13000D012,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0078,-0.0142,-0.0617,-0.3327,-1.0902,-4.8055,-0.2074,-0.0000,0.0000,0.0029,0.0035,0.0378,0.3605,2.2492,22.6080,0.0553,0.0000
PEP13000M088,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0013,-0.0004,-0.0147,-0.0501,-0.2770,-2.4626,-3.0764,-0.0000,-0.0000,0.0005,0.0000,0.0096,0.0368,0.3081,4.4240,8.1440,0.0000,0.0000


### 4.2 Calculating the portfolio that maximizes Sharpe ratio.

In [14]:
########################################################################
# Calculating optimum portfolio
########################################################################

port = rp.Portfolio(returns=Y)

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

port.factors = X
port.factors_stats(method_mu=method_mu, method_cov=method_cov, d=0.94, B=loadings)

# Estimate optimal portfolio:

model='FM' # Factor Model
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.style.format("{:.4%}").background_gradient(cmap='YlGn'))

,weights
APA,0.0000%
CMCSA,13.3018%
CNP,10.5311%
HPQ,15.1020%
PSA,26.1340%
SEE,0.0000%
ZION,8.2247%
PEP11900D031,0.0000%
PEP13000D012,0.0000%
PEP13000M088,0.0000%


## 5. Optimization of Equity and Bond Portfolio with Key Rate Durations Constraints

This part shows how __Riskfolio-Lib__ can be used to build immunized portfolios using __duration matching__ and __convexity matching__, however the example only use duration matching. More information about inmunization theory can be found in this __[link](https://www.investopedia.com/terms/i/immunization.asp)__.

### 5.1 Statistics of Risk Factors

In [15]:
########################################################################
# Displaying factors statistics
########################################################################

table = pd.concat([loadings.min(), loadings.max()], axis=1)
table.columns = ['min', 'max']
display(table.iloc[:16,:].style.format("{:.4f}").background_gradient(cmap='YlGn'))
display(X.iloc[:,:16].corr().style.format("{:.4f}").background_gradient(cmap='YlGn'))

,min,max
APA,0.0000,1.0000
CMCSA,0.0000,1.0000
CNP,0.0000,1.0000
HPQ,0.0000,1.0000
PSA,0.0000,1.0000
SEE,0.0000,1.0000
ZION,0.0000,1.0000
R 0,-0.0015,-0.0000
R 90,-0.0078,0.0000
R 180,-0.0192,0.0000


,APA,CMCSA,CNP,HPQ,PSA,SEE,ZION,R 0,R 90,R 180,R 360,R 720,R 1800,R 3600,R 7200,R 10800
APA,1.0000,0.2775,0.3307,0.3190,0.0872,0.2098,0.4060,-0.0059,-0.0057,-0.0020,-0.0113,-0.0428,-0.0662,-0.0420,-0.0141,-0.0106
CMCSA,0.2775,1.0000,0.2973,0.2571,0.2188,0.3106,0.3600,-0.0095,-0.0225,-0.0377,-0.0529,-0.0456,-0.0376,-0.0455,-0.0314,-0.0162
CNP,0.3307,0.2973,1.0000,0.2581,0.3473,0.3289,0.1952,-0.0126,-0.0070,-0.0040,-0.0200,-0.0571,-0.0866,-0.0768,-0.0530,-0.0503
HPQ,0.3190,0.2571,0.2581,1.0000,0.1820,0.3341,0.3727,-0.0395,-0.0428,-0.0408,-0.0364,-0.0454,-0.0829,-0.0792,-0.0582,-0.0553
PSA,0.0872,0.2188,0.3473,0.1820,1.0000,0.2633,0.0666,-0.0067,0.0057,0.0096,0.0034,-0.0105,-0.0414,-0.0674,-0.0659,-0.0602
SEE,0.2098,0.3106,0.3289,0.3341,0.2633,1.0000,0.3617,-0.0183,-0.0339,-0.0396,-0.0267,0.0075,0.0167,-0.0176,-0.0414,-0.0476
ZION,0.4060,0.3600,0.1952,0.3727,0.0666,0.3617,1.0000,-0.0185,-0.0267,-0.0228,-0.0148,-0.0073,0.0049,0.0199,0.0274,0.0215
R 0,-0.0059,-0.0095,-0.0126,-0.0395,-0.0067,-0.0183,-0.0185,1.0000,0.9233,0.7183,0.2411,-0.0500,0.0302,0.0256,0.0284,0.0489
R 90,-0.0057,-0.0225,-0.0070,-0.0428,0.0057,-0.0339,-0.0267,0.9233,1.0000,0.9197,0.5137,0.0981,0.0442,0.0441,0.0441,0.0523
R 180,-0.0020,-0.0377,-0.0040,-0.0408,0.0096,-0.0396,-0.0228,0.7183,0.9197,1.0000,0.7901,0.3509,0.1002,0.0579,0.0700,0.0732


### 5.2 Creating Constraints on Key Rate Durations

In this example we are going to put a limit on the maximum duration that the portfolio can reach. The key rate durations of portfolio for 1800, 3600 and 7200 days will be lower than -2, -2 and -3.

In [16]:
########################################################################
# Creating key rate durations constraints
########################################################################

constraints = {'Disabled': [False, False, False],
               'Factor': ['R 1800', 'R 3600', 'R 7200'],
               'Sign': ['<=', '<=', '<='],
               'Value': [-2, -2, -3],
               'Relative Factor': ['', '', '']}

constraints = pd.DataFrame(constraints)

display(constraints)

,Disabled,Factor,Sign,Value,Relative Factor
0,False,R 1800,<=,-2,
1,False,R 3600,<=,-2,
2,False,R 7200,<=,-3,


### 5.3 Estimating Optimum Portfolio with Key Rate Durations Constraints

In [17]:
########################################################################
# Estimating optimum portfolio with key rate durations constraints
########################################################################

C, D = rp.factors_constraints(constraints, loadings)

port.ainequality = C
port.binequality = D

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.style.format("{:.4%}").background_gradient(cmap='YlGn'))

,weights
APA,0.0000%
CMCSA,0.0000%
CNP,0.0000%
HPQ,5.3862%
PSA,0.0000%
SEE,0.0000%
ZION,0.0000%
PEP11900D031,0.0000%
PEP13000D012,0.0000%
PEP13000M088,0.0000%


In [18]:
########################################################################
# Calculating portfolio sensitivities for each risk factor
########################################################################

d_ = np.matrix(loadings).T * np.matrix(w)
d_ = pd.DataFrame(d_, index=loadings.columns, columns=['Values'])

display(d_.style.format("{:.4f}").background_gradient(cmap='YlGn'))

,Values
APA,0.0000
CMCSA,0.0000
CNP,0.0000
HPQ,0.0539
PSA,0.0000
SEE,0.0000
ZION,0.0000
R 0,-0.0006
R 90,-0.0030
R 180,-0.0109


We can see that this portfolio fulfill all constraints on key rate durations.